# A Tiny Temperature Converter Library
## Written in Julia (a naive approach)

Based on a wonderful example from _Erik Engheim_

We start with some type definitions!

In [1]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Kelvin <: Temperature
   value::Float64 
end

struct Fahrenheit <: Temperature
    value::Float64
end

Define the promotion rules, so Julia knows our preferences when mixing types.

In [2]:
import Base: promote_rule  # we import the `promote_rule` function to add our own methods

promote_rule(::Type{Kelvin}, ::Type{Celsius})     = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin})  = Kelvin

promote_rule (generic function with 126 methods)

We implement the conversion logic by adding methods to the `convert` function, available in the `Base` of Julia.

In [3]:
import Base: convert  # again, we add our own methods to `convert`

convert(::Type{Kelvin},  t::Celsius)     = Kelvin(t.value + 273.15)
convert(::Type{Kelvin},  t::Fahrenheit)  = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin)      = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t.value*9/5 + 32)
convert(::Type{Fahrenheit}, t::Kelvin)   = Fahrenheit(Celsius(t))

convert (generic function with 730 methods)

We can add some nice constructors, so we can initialise each 

In [4]:
Kelvin(t::Temperature) = convert(Kelvin, t)
Celsius(t::Temperature) = convert(Celsius, t)
Fahrenheit(t::Temperature) = convert(Fahrenheit, t)

Fahrenheit

In [5]:
Kelvin(Fahrenheit(23))

Kelvin(268.15)

Adding arithmetic operations for our types by extending the `Base` operators.

In [6]:
import Base: +, -, *  # operators are functions, we can extend them easily!

+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)

+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)

*(x::Number, y::T) where {T <: Temperature} = T(x * y.value);

Let's test it:

In [7]:
Celsius(37) + Kelvin(10)

Kelvin(320.15)

In [8]:
Celsius(Celsius(37) + Kelvin(10))

Celsius(47.0)

In [9]:
Fahrenheit(23) + Celsius(42) - Kelvin(5)  # this does not need to make sense now ;)

Kelvin(305.15)

A nice way to create some syntactic sugar:

In [10]:
const °C = Celsius(1)
const °F = Fahrenheit(1)
const K = Kelvin(1);

In [11]:
5°F, 23K, 42°C

(Fahrenheit(5.0), Kelvin(23.0), Celsius(42.0))

In [12]:
42°C + 10K

Kelvin(325.15)

### Alright, let's have a look behind the scenes...

In [13]:
example_calculation() = Fahrenheit(2) + Celsius(3) + Kelvin(4)

example_calculation (generic function with 1 method)

In [14]:
@code_llvm example_calculation()


define %Kelvin @julia_example_calculation_62940() #0 !dbg !5 {
top:
  ret %Kelvin { double 0x407077BBBBBBBBBB }
}


LLVM figured out that the function returns always the same value... Dooh ;)

This is the "machine code":

In [15]:
@code_native example_calculation()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[13]
	pushl	%ebp
	decl	%eax
	movl	%esp, %ebp
	decl	%eax
	movl	$478589824, %eax        ## imm = 0x1C86B380
	addl	%eax, (%eax)
	addb	%al, (%eax)
Source line: 1
	movsd	(%eax), %xmm0           ## xmm0 = mem[0],zero
	popl	%ebp
	retl
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop


Now a function which is not constant ;)

In [16]:
another_calculation(a, b, c) = Fahrenheit(a) + Celsius(b) + Kelvin(c)

another_calculation (generic function with 1 method)

In [17]:
@code_llvm another_calculation(1, 2, 3)


define %Kelvin @julia_another_calculation_62960(i64, i64, i64) #0 !dbg !5 {
top:
  %3 = sitofp i64 %0 to double
  %4 = fadd double %3, -3.200000e+01
  %5 = fmul double %4, 5.000000e+00
  %6 = fdiv double %5, 9.000000e+00
  %7 = sitofp i64 %1 to double
  %8 = fadd double %7, %6
  %9 = fadd double %8, 2.731500e+02
  %10 = sitofp i64 %2 to double
  %11 = fadd double %10, %9
  %12 = insertvalue %Kelvin undef, double %11, 0
  ret %Kelvin %12
}


And the machine code:

In [18]:
@code_native another_calculation(2.0, 3.0, 4.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[16]
	pushl	%ebp
	decl	%eax
	movl	%esp, %ebp
	decl	%eax
	movl	$478594312, %eax        ## imm = 0x1C86C508
	addl	%eax, (%eax)
	addb	%al, (%eax)
Source line: 1
	addsd	(%eax), %xmm0
	decl	%eax
	movl	$478594320, %eax        ## imm = 0x1C86C510
	addl	%eax, (%eax)
	addb	%al, (%eax)
	mulsd	(%eax), %xmm0
	decl	%eax
	movl	$478594328, %eax        ## imm = 0x1C86C518
	addl	%eax, (%eax)
	addb	%al, (%eax)
	divsd	(%eax), %xmm0
	addsd	%xmm1, %xmm0
	decl	%eax
	movl	$478594336, %eax        ## imm = 0x1C86C520
	addl	%eax, (%eax)
	addb	%al, (%eax)
	addsd	(%eax), %xmm0
	addsd	%xmm2, %xmm0
	popl	%ebp
	retl
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop
	nop


## What about the actual performance?

In [19]:
using BenchmarkTools

In [20]:
@benchmark another_calculation(2.0, 3.0, 4.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.901 ns (0.00% GC)
  median time:      1.903 ns (0.00% GC)
  mean time:        2.059 ns (0.00% GC)
  maximum time:     32.996 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

We can easily define our own `rand` method for `Kelvin` temperatures ()

In [21]:
import Base: rand

rand(::MersenneTwister, ::Type{Kelvin}) = Kelvin(rand() * 5000)

temperatures = rand(Kelvin, 1_000_000)  # one million values in Kelvin

1000000-element Array{Kelvin,1}:
 Kelvin(786.699)
 Kelvin(3526.04)
 Kelvin(2836.04)
 Kelvin(1070.38)
 Kelvin(2469.57)
 Kelvin(1652.41)
 Kelvin(561.549)
 Kelvin(4913.06)
 Kelvin(1774.79)
 Kelvin(478.199)
 Kelvin(349.99) 
 Kelvin(1994.88)
 Kelvin(4737.78)
 ⋮              
 Kelvin(1455.66)
 Kelvin(1745.72)
 Kelvin(3667.81)
 Kelvin(2814.69)
 Kelvin(158.547)
 Kelvin(482.548)
 Kelvin(386.658)
 Kelvin(525.919)
 Kelvin(3973.32)
 Kelvin(4017.23)
 Kelvin(547.849)
 Kelvin(2464.27)

### Is this fast?

In [22]:
@benchmark rand(Kelvin, 1_000_000)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     8.569 ms (0.00% GC)
  median time:      10.443 ms (0.00% GC)
  mean time:        11.180 ms (10.39% GC)
  maximum time:     73.797 ms (87.44% GC)
  --------------
  samples:          447
  evals/sample:     1

### Yep... 10ms, 2 allocs, 8 MiB memory

### Convert them to `Celsius` by using our type constructor and the `.`-notation for element-wise operation (similar to Matlab or `ufuncs` in numpy)

In [23]:
Celsius.(temperatures)

1000000-element Array{Celsius,1}:
 Celsius(513.549) 
 Celsius(3252.89) 
 Celsius(2562.89) 
 Celsius(797.23)  
 Celsius(2196.42) 
 Celsius(1379.26) 
 Celsius(288.399) 
 Celsius(4639.91) 
 Celsius(1501.64) 
 Celsius(205.049) 
 Celsius(76.8398) 
 Celsius(1721.73) 
 Celsius(4464.63) 
 ⋮                
 Celsius(1182.51) 
 Celsius(1472.57) 
 Celsius(3394.66) 
 Celsius(2541.54) 
 Celsius(-114.603)
 Celsius(209.398) 
 Celsius(113.508) 
 Celsius(252.769) 
 Celsius(3700.17) 
 Celsius(3744.08) 
 Celsius(274.699) 
 Celsius(2191.12) 

### Is this fast?

In [24]:
@benchmark Celsius.(temperatures)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  26
  --------------
  minimum time:     2.411 ms (0.00% GC)
  median time:      3.805 ms (0.00% GC)
  mean time:        4.762 ms (27.86% GC)
  maximum time:     75.720 ms (96.37% GC)
  --------------
  samples:          1044
  evals/sample:     1

### Yep... 2.7ms, 26 allocs, 8 MiB memory